In [1]:
from tensorflow.keras.applications import vgg19
from tensorflow.keras.preprocessing.image import load_img, save_img, img_to_array
from tensorflow.keras import backend as K
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time

In [2]:
img_nrows = 224
img_ncols = 224
base_image_path = 'origin_2.jpg'
style_reference_image_path = 'style_3.jpg'
content_weight = 0.025
style_weight = 1
total_variation_weight = 1
iterations = 10

In [3]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

In [4]:
# get tensor representations of our images
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [6]:
model = vgg19.VGG19(input_tensor=input_tensor, weights='imagenet')

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])

In [7]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (3, 224, 224, 3)          0         
_________________________________________________________________
block1_conv1 (Conv2D)        (3, 224, 224, 64)         1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (3, 224, 224, 64)         36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (3, 112, 112, 64)         0         
_________________________________________________________________
block2_conv1 (Conv2D)        (3, 112, 112, 128)        73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (3, 112, 112, 128)        147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (3, 56, 56, 128)          0         
__________

In [8]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1))) #channel first
    gram = K.dot(features, K.transpose(features))
    return gram

In [9]:
def style_loss(style, combination):
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [10]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))

In [11]:
def total_variation_loss(x):
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :]) #x_dimension
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :]) #y_dimension
    return K.sum(K.pow(a + b, 1.25))

loss = K.variable(0.0)

loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [12]:

layer_features = outputs_dict['block5_conv2']

base_image_features = layer_features[0, :, :, :] # input.shape = [base, style, combination]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                      combination_features)

In [13]:
feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']

for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] # input.shape = [base, style, combination]
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl # mean of all output extract

In [14]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
outputs += grads #append tuple

f_outputs = K.function([combination_image], outputs)


In [15]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

In [16]:
class Evaluator(object):
    def __init__(self):
        self.loss_value = None
        self.grads_values = None
    def loss(self, x):
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value
    def grads(self, x):
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

In [17]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
evaluator = Evaluator()
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = 'res8_at_iteration_%d.png' % i
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

Start of iteration 0
Current loss value: 13938958000.0
Image saved as res8_at_iteration_0.png
Iteration 0 completed in 44s
Start of iteration 1
Current loss value: 6796858000.0
Image saved as res8_at_iteration_1.png
Iteration 1 completed in 42s
Start of iteration 2
Current loss value: 5045186000.0
Image saved as res8_at_iteration_2.png
Iteration 2 completed in 42s
Start of iteration 3
Current loss value: 4323467000.0
Image saved as res8_at_iteration_3.png
Iteration 3 completed in 44s
Start of iteration 4
Current loss value: 3832209000.0
Image saved as res8_at_iteration_4.png
Iteration 4 completed in 45s
Start of iteration 5
Current loss value: 3572564200.0
Image saved as res8_at_iteration_5.png
Iteration 5 completed in 45s
Start of iteration 6
Current loss value: 3422592800.0
Image saved as res8_at_iteration_6.png
Iteration 6 completed in 49s
Start of iteration 7
Current loss value: 3279102200.0
Image saved as res8_at_iteration_7.png
Iteration 7 completed in 45s
Start of iteration 8
Cu